In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
from itertools import combinations
from collections import Counter

In [3]:
import sys
sys.path.append('..')

from util.participant_stream import ParticipantStream

# Algo 1
Greedy algorithm that weights samples with lower probability greater
1. randomly sample from participant pool and build distribution
2. compute kde over this space
3. for each sample, assign weight equals to probability of sample from kde
4. assign in order from lowest probability first

## Basic Proof of Concept
For first run, we will ignore the arrival times and assume we have full pool available and see how the algorithm does assuming perfect information of pool to get a baseline.

In [4]:
rng = np.random.default_rng(3098)

In [5]:
stream = ParticipantStream(rng=rng)
pool = stream.generate_participants(5000)

In [6]:
# sample from pool to build distribution
ages = np.fromiter((p['age'] for p in pool), dtype=int)
sampled_pairs = rng.choice(ages, (2, 10000))
sampled_pairs

array([[40, 32, 53, ..., 47, 44, 38],
       [48, 58, 46, ..., 24, 47, 42]])

In [7]:
sample_p = stats.gaussian_kde(sampled_pairs)

In [8]:
test = sample_p.resample(1000, seed=rng)
test

array([[30.43603239, 61.9449133 , 45.39398149, ..., 48.98592035,
        19.28995238, 57.73759347],
       [47.52676105, 36.52766666, 59.60344411, ..., 37.95646493,
        37.68361473, 60.22914472]])

In [9]:
all_pairs = combinations(ages, 2)
all_pairs = list(all_pairs)
all_pairs[:5]

[(48, 32), (48, 40), (48, 46), (48, 38), (48, 38)]

In [10]:
all_pairs_prob = sample_p(np.array(all_pairs).transpose())

In [11]:
true_best = all_pairs[np.argsort(all_pairs_prob)]
true_best

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
for i in range(10):
    print(true_best[i], sample_p[true_best[i]])

In [ ]:
def greedy_pairs(pair_list, pool):
    age_map = {}
    for p in pool:
        if p['age'] not in age_map:
            age_map[p['age']] = [p]
        else:
            age_map[p['age']].append(p)
        grouping = []
    for pair in pair_list:
        p1 = int(pair[0])
        p2 = int(pair[1])
        if p1 > p2:
            p1, p2 = p2, p1
        if p1 == p2 and len(age_map[p1]) < 2:
            continue
        if p1 in age_map and p2 in age_map:
            grouping = (age_map[p1].pop(), age_map[p2].pop())
            if len(age_map[p1]) == 0:
                del age_map[p1]
            if len(age_map[p2]) == 0:
                del age_map[p2]
            if not age_map:
                break
        else:
            print(p1, p2, 'not found')
    return grouping

In [ ]:
true_grouping = greedy_pairs(true_best, pool)

In [ ]:
test_prob = sample_p(test)
test_prob

array([1.23828741e-04, 3.38118577e-05, 8.56033336e-04, 4.69102473e-04,
       5.73429739e-04, 4.66349494e-05, 3.62634733e-04, 1.21100294e-04,
       8.10682351e-04, 5.98632733e-04, 8.74528614e-05, 5.32599731e-04,
       4.14081652e-04, 2.60361177e-04, 4.87726287e-04, 5.25174383e-05,
       4.43686666e-04, 4.25004727e-04, 1.69032063e-04, 3.28735755e-05,
       2.71986983e-04, 3.98263653e-04, 7.78604733e-04, 3.42961058e-04,
       3.21091929e-04, 8.38445129e-04, 3.19931680e-04, 5.24975869e-05,
       7.96640633e-04, 8.60986103e-04, 6.35999666e-04, 4.87021299e-04,
       2.14447571e-05, 6.61552206e-04, 4.05144200e-04, 6.13690520e-04,
       1.96450330e-04, 2.92254567e-04, 7.00096779e-04, 4.13425406e-04,
       1.90949843e-04, 7.21428081e-04, 3.40300246e-04, 6.17631172e-04,
       6.87754209e-04, 3.11126888e-04, 2.85793341e-04, 4.99289411e-04,
       2.70377191e-04, 7.71349012e-04, 7.56533502e-04, 5.62999700e-04,
       8.55391649e-05, 3.42953205e-04, 1.51379843e-04, 5.80134350e-05,
      

In [ ]:
test_pairs = test.transpose()[np.argsort(test_prob)]
test_pairs

array([[12.97122186, 30.09865375],
       [18.93179215, 15.26109957],
       [81.052842  , 27.80997453],
       ...,
       [41.72029639, 43.40337026],
       [41.28158132, 44.33764653],
       [41.47143483, 44.81627961]])

In [ ]:
test_grouping = greedy_pairs(test_pairs, pool)
test_grouping

12 30 not found
15 18 not found
27 81 not found
36 82 not found
15 64 not found
15 44 not found
42 80 not found
15 35 not found
16 27 not found
16 27 not found
44 80 not found
48 80 not found
16 31 not found
16 53 not found
17 26 not found
16 38 not found
16 49 not found
16 38 not found
16 36 not found
16 39 not found
16 37 not found
16 38 not found
18 61 not found
17 34 not found
17 40 not found
18 52 not found
17 36 not found
18 61 not found
17 39 not found
18 45 not found
18 49 not found
18 44 not found
18 45 not found
18 39 not found


({'party': 'Independent',
  'age': 41,
  'gender': 'M',
  'arrival_time': 24,
  'departure_time': 45},
 {'party': 'Democrat',
  'age': 44,
  'gender': 'M',
  'arrival_time': 22,
  'departure_time': 65})

We try a different strategy. Build KDE of pool and just greedily select participant with least probability of showing up based on kde. This is easier to compute and will probably scale better as the dimensions of the kde is much smaller.

In [ ]:
pop_p = stats.gaussian_kde(ages)

In [ ]:
rareness = pop_p(ages)
rareness

array([0.02740866, 0.01876117, 0.02817543, ..., 0.02680299, 0.02902373,
       0.02153773])

In [ ]:
ranked_idx = np.argsort(rareness)
ranked_idx

array([1469, 3684, 4559, ...,  708, 1555, 4803], dtype=int64)

In [ ]:
groupings = []
for i in range(0, len(pool), 2):
    i1 = ranked_idx[i]
    i2 = ranked_idx[i+1]
    if pool[i1]['age'] < pool[i2]['age']:
        groupings.append((pool[i1], pool[i2]))
    else:
        groupings.append((pool[i2], pool[i1]))
groupings

[({'party': 'Democrat',
   'age': 79,
   'gender': 'F',
   'arrival_time': 38,
   'departure_time': 48},
  {'party': 'Republican',
   'age': 79,
   'gender': 'F',
   'arrival_time': 12,
   'departure_time': 22}),
 ({'party': 'Democrat',
   'age': 79,
   'gender': 'M',
   'arrival_time': 29,
   'departure_time': 114},
  {'party': 'Democrat',
   'age': 79,
   'gender': 'F',
   'arrival_time': 79,
   'departure_time': 84}),
 ({'party': 'Democrat',
   'age': 79,
   'gender': 'M',
   'arrival_time': 27,
   'departure_time': 36},
  {'party': 'Democrat',
   'age': 79,
   'gender': 'M',
   'arrival_time': 15,
   'departure_time': 35}),
 ({'party': 'Republican',
   'age': 79,
   'gender': 'F',
   'arrival_time': 20,
   'departure_time': 41},
  {'party': 'Republican',
   'age': 79,
   'gender': 'F',
   'arrival_time': 54,
   'departure_time': 73}),
 ({'party': 'Democrat',
   'age': 79,
   'gender': 'F',
   'arrival_time': 6,
   'departure_time': 64},
  {'party': 'Republican',
   'age': 79,
   'g

In [ ]:
len(groupings)

2500